In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi
from pyspark.sql import SparkSession
from datetime import date

#create API connection and authenticate
api = KaggleApi()
api.authenticate()

#download dataset 
api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Daily_Country_Wise_Confirmed_Cases.csv')

#create spark session
spark = SparkSession.builder.getOrCreate()

#create spark df
daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

def date_check():
    today = date.today()
    return str(today)

def column_grab():
    try:
        return daily_df.select(daily_df[date_check()]).show()
    except:
        pass

column_grab()

In [ ]:
from airflow import DAG
from airflow.decorators import dag, task

#create our tasks
@task
def api_pull():
    """
    pulls the updated file from Kaggle
    """
    api = KaggleApi()
    api.authenticate()
    #download dataset 
    api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Daily_Country_Wise_Confirmed_Cases.csv')
    #create spark session
    spark = SparkSession.builder.getOrCreate()
    #create spark df
    daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

@task
def main():
    """
    main.py, update table with new data, do transformations
    """

@task
def load():
    """
    load to bigqquery
    """

#set dag attributes
@dag(
    schedule_interval='@daily',
    start_date=datetime.utcnow(),
    catchup=False,
)
#function that calls all of our tasks to create dag
def monkeypox_dag():
    

